In [20]:
import pandas as pd

train_df = pd.read_csv('../data/train.csv')  
test_df = pd.read_csv('../data/test.csv')

print(train_df.head())  

print(train_df.info())

# Kiểm tra số lượng giá trị bị thiếu
print(train_df.isnull().sum().sort_values(ascending=False)[:20])  # Hiển thị 20 cột có nhiều giá trị thiếu nhất


   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [ ]:
numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns
print("Numerical Features:", numerical_features)

Numerical Features: Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')


In [5]:
categorical_features = train_df.select_dtypes(include=['object']).columns
print("Categorical Features:", categorical_features)

Categorical Features: Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')


In [6]:
# Select 2 continuous features
selected_continuous = ['GrLivArea', 'LotArea']

# Select 2 categorical features
selected_categorical = ['Neighborhood', 'HouseStyle']

In [23]:
from sklearn.preprocessing import StandardScaler

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
train_cont_scaled = scaler.fit_transform(train_df[selected_continuous])


In [27]:
from sklearn.preprocessing import OneHotEncoder

# Mã hóa dữ liệu phân loại (Sửa lỗi `sparse` -> `sparse_output`)
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Áp dụng OneHotEncoder cho các cột phân loại
train_cat_encoded = encoder.fit_transform(train_df[selected_categorical])


In [28]:
import numpy as np

# Kết hợp đặc trưng liên tục và phân loại
X_train = np.hstack((train_cont_scaled, train_cat_encoded))
y_train = train_df['SalePrice']


In [29]:
from sklearn.model_selection import train_test_split

X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [30]:
from sklearn.linear_model import LinearRegression

# Khởi tạo mô hình
model = LinearRegression()

# Huấn luyện mô hình
model.fit(X_train_split, y_train_split)


LinearRegression()

In [31]:
import numpy as np
from sklearn.metrics import mean_squared_log_error

def compute_rmsle(y_test, y_pred, precision=2):
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_val)

# Tính RMSLE
error = compute_rmsle(y_val, y_pred)
print(f'RMSLE: {error}')


RMSLE: 0.2
